In [10]:
import os
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
from langchain_community.agent_toolkits import FileManagementToolkit

import json


def get_ddgSearch(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    keyword = inputs["keyword"]
    return ddg.run(f"research of {keyword}")


def get_wikiSearch(inputs):
    wiki = WikipediaAPIWrapper()
    keyword = inputs["keyword"]
    return wiki.run(f"research of {keyword}")


def save_to_file(query, result):
    working_directory = os.getcwd()
    tools = FileManagementToolkit(
        root_dir=working_directory,
        selected_tools=["write_file"],
    ).get_tools()

    (write_tool,) = tools

    results = write_tool.invoke({"file_path": f"{query}.txt", "text": result["output"]})
    return results


functions_map = {
    "get_ddgSearch": get_ddgSearch,
    "get_wikiSearch": get_wikiSearch,
    "save_to_file": save_to_file,
}

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ddgSearch",
            "description": "find the duckduck go search Result in this website with keyword arugment.",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "The research of keyword",
                    }
                },
                "required": ["keyword"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_wikiSearch",
            "description": "find the wikipedia search Result in this website with query arugment.",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "The research of keyword",
                    }
                },
                "required": ["keyword"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "save_to_file",
            "description": "Save information to a file.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The query used to retrieve the information.",
                    },
                    "result": {
                        "type": "string",
                        "description": "The result to be saved in the file.",
                    },
                },
                "required": ["query", "result"],
            },
        },
    },
]

In [11]:
import openai as client

#assistant = client.beta.assistants.create(
#    name="Research AI Assistant",
#    instructions="""You are a helful search manager assistance.
            
#            You should try to search information WikiPedia or DuckDuckGo,
            
#            If there is a list of website urls in the search results list, please extract the content of each website to text and organize it, 
#            This will be documented, so please keep the results organized""",
#    model="gpt-4-1106-preview",
#    tools=functions,
#)
assistant_id = "asst_md3AYOa4IsJtVFEl9bTitIe2"

In [12]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want to know if the Salesforce stock is a good buy",
        }
    ]
)
thread

Thread(id='thread_nlIHJ5QIm5ppFr1sAWegSMSu', created_at=1713110532, metadata={}, object='thread')

In [13]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

Run(id='run_XuL7DO5m1TFlo0Vpkkg9m5vu', assistant_id='asst_md3AYOa4IsJtVFEl9bTitIe2', cancelled_at=None, completed_at=None, created_at=1713110534, expires_at=1713111134, failed_at=None, file_ids=[], instructions='You are a helful search manager assistance.\n            \n            You should try to search information WikiPedia or DuckDuckGo,\n            \n            If there is a list of website urls in the search results list, please extract the content of each website to text and organize it, \n            This will be documented, so please keep the results organized', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_nlIHJ5QIm5ppFr1sAWegSMSu', tools=[FunctionTool(function=FunctionDefinition(name='get_ddgSearch', description='find the duckduck go search Result in this website with keyword arugment.', parameters={'type': 'object', 'properties': {'keyword': {'type': 'string', 'desc

In [14]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id).data
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")


def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread.id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outpus = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id, thread_id=thread_id, tool_outputs=outpus
    )

In [16]:
get_run(run.id, thread.id)

Run(id='run_XuL7DO5m1TFlo0Vpkkg9m5vu', assistant_id='asst_md3AYOa4IsJtVFEl9bTitIe2', cancelled_at=None, completed_at=None, created_at=1713110534, expires_at=1713111134, failed_at=None, file_ids=[], instructions='You are a helful search manager assistance.\n            \n            You should try to search information WikiPedia or DuckDuckGo,\n            \n            If there is a list of website urls in the search results list, please extract the content of each website to text and organize it, \n            This will be documented, so please keep the results organized', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_BGq2ML2VCQeNjSfsfieYFXrv', function=Function(arguments='{"keyword":"is Salesforce stock a good buy"}', name='get_ddgSearch'), type='function')]), type='submit_tool_outputs'), started_at=1713110534, status=

In [17]:
submit_tool_outputs(run.id, thread.id)

Calling function: get_ddgSearch with arg {"keyword":"is Salesforce stock a good buy"}


Run(id='run_XuL7DO5m1TFlo0Vpkkg9m5vu', assistant_id='asst_md3AYOa4IsJtVFEl9bTitIe2', cancelled_at=None, completed_at=None, created_at=1713110534, expires_at=1713111134, failed_at=None, file_ids=[], instructions='You are a helful search manager assistance.\n            \n            You should try to search information WikiPedia or DuckDuckGo,\n            \n            If there is a list of website urls in the search results list, please extract the content of each website to text and organize it, \n            This will be documented, so please keep the results organized', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=1713110534, status='queued', thread_id='thread_nlIHJ5QIm5ppFr1sAWegSMSu', tools=[FunctionTool(function=FunctionDefinition(name='get_ddgSearch', description='find the duckduck go search Result in this website with keyword arugment.', parameters={'type': 'object', 'properties': {'keyword': {'type': 'string',

In [18]:
get_run(run.id,thread.id).status

'completed'

In [19]:
get_messages(thread.id)

user: I want to know if the Salesforce stock is a good buy
assistant: Based on the search results, here's the summary of opinions regarding Salesforce stock as a buy:

- Salesforce stock is considered *fairly valued* with a 3-star rating, according to a source that provided a long-term fair value estimate of $255 per share. 
- Brent Thill, an analyst from **Jefferies**, has maintained a *Buy* rating on Salesforce with a price target of $350.00.
- Salesforce is mentioned as a candidate for a **"buy-the-dip"** strategy, with a notable rally of nearly 60% in 2023 following a 48% decline in 2022.
- Investors often look for **Wall Street analysts' recommendations** when considering whether to buy, sell, or hold a stock.

No direct advice on whether Salesforce stock is a definitive buy was provided in the search results. However, the information implies that some analysts maintain a positive outlook on the stock with a buy rating and an expectation of a rebound in value after previous declin